# XML Agent

Some language models (like Anthropic's Claude) are particularly good at reasoning/writing XML. This goes over how to use an agent that uses XML when prompting. 

## Initialize the tools

We will initialize some fake tools for demo purposes

In [1]:
from langchain.agents import tool

@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

In [6]:
tools = [search]

In [2]:
from langchain.chat_models import ChatAnthropic
model = ChatAnthropic(model="claude-2")

## Use LangChain Expression Language

We will first show how to create this agent using LangChain Expression Language

In [4]:
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain.agents.format_scratchpad import format_xml
from langchain import hub

In [3]:
prompt = hub.pull("hwchase17/xml-agent")

In [7]:
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [9]:
llm_with_stop = model.bind(stop=["</tool_input>"])

In [15]:
agent = {
    "question": lambda x: x["question"],
    "agent_scratchpad": lambda x: format_xml(x['intermediate_steps']),
} | prompt | llm_with_stop | XMLAgentOutputParser()

In [ ]:
from langchain.agents import AgentExecutor

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.invoke({"question": "whats the weather in New york?"})



> Entering new AgentExecutor chain...
 <tool>search</tool>
<tool_input>weather in new york32 degrees <tool>search</tool>
<tool_input>weather in new york32 degrees <final_answer>
The weather in New York is 32 degrees.
</final_answer>

> Finished chain.


{'question': 'whats the weather in New york?',
 'output': '\nThe weather in New York is 32 degrees.\n'}

## Use off-the-shelf agent

In [22]:
from langchain.chains import LLMChain
from langchain.agents import XMLAgent

In [23]:
chain = LLMChain(
    llm=model,
    prompt=XMLAgent.get_default_prompt(),
    output_parser=XMLAgent.get_default_output_parser()
)
agent = XMLAgent(tools=tools, llm_chain=chain)

In [25]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
agent_executor.invoke({"input": "whats the weather in New york?"})



> Entering new AgentExecutor chain...
 <tool>search</tool>
<tool_input>weather in new york32 degrees

<final_answer>The weather in New York is 32 degrees

> Finished chain.


{'input': 'whats the weather in New york?',
 'output': 'The weather in New York is 32 degrees'}